<a href="https://colab.research.google.com/github/ikyath/M5-Forecasting-Accuracy-Kaggle/blob/master/M5Forecasting%26Accuracy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Abstract

Welcome to the "M5 Forecasting - Accuracy" competition! In this competition, contestants are challenged to forecast future sales at Walmart based on heirarchical sales in the states of California, Texas, and Wisconsin. Forecasting sales, revenue, and stock prices is a classic application of machine learning in economics, and it is important because it allows investors to make guided decisions based on forecasts made by algorithms

In [1]:
pwd

'/content'

In [2]:
cd /content/drive/My\ Drive/Data\ Science

/content/drive/My Drive/Data Science


In [0]:
import numpy as np
import pandas as pd
import joblib
from tqdm.notebook import tqdm as tqdm
from sklearn.preprocessing import OrdinalEncoder
import matplotlib.pyplot as plt

import gc

In [0]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [0]:
def read_data(PATH):
    print('Reading files...')
    calendar = pd.read_csv(f'{PATH}/calendar.csv')
    calendar = reduce_mem_usage(calendar)
    print('Calendar has {} rows and {} columns'.format(calendar.shape[0], calendar.shape[1]))
    sell_prices = pd.read_csv(f'{PATH}/sell_prices.csv')
    sell_prices = reduce_mem_usage(sell_prices)
    print('Sell prices has {} rows and {} columns'.format(sell_prices.shape[0], sell_prices.shape[1]))
    sales_train_validation = pd.read_csv(f'{PATH}/sales_train_validation.csv')
    print('Sales train validation has {} rows and {} columns'.format(sales_train_validation.shape[0], sales_train_validation.shape[1]))
    submission = pd.read_csv(f'{PATH}/sample_submission.csv')
    return calendar, sell_prices, sales_train_validation, submission

In [6]:
calendar, sell_prices, sales_train_validation, submission = read_data("/content/drive/My Drive/Data Science")

Reading files...
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Calendar has 1969 rows and 14 columns
Mem. usage decreased to 130.48 Mb (37.5% reduction)
Sell prices has 6841121 rows and 4 columns
Sales train validation has 30490 rows and 1919 columns


In [7]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,d_32,d_33,d_34,...,d_1874,d_1875,d_1876,d_1877,d_1878,d_1879,d_1880,d_1881,d_1882,d_1883,d_1884,d_1885,d_1886,d_1887,d_1888,d_1889,d_1890,d_1891,d_1892,d_1893,d_1894,d_1895,d_1896,d_1897,d_1898,d_1899,d_1900,d_1901,d_1902,d_1903,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,1,3,1,3,1,2,2,0,1,1,1,1,0,0,0,0,0,1,0,4,2,3,0,1,2,0,0,0,1,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,2,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,3,4,2,1,4,1,3,5,0,6,6,0,0,0,0,3,1,2,1,3,1,0,2,5,4,2,0,3,0,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1,0,3,2,2,2,3,1,0,0,0,0,1,0,4,4,0,1,4,0,1,0,1,0,1,1,2,0,1,1,2,1,1,0,1,1,2,2,2,4


In [8]:
calendar.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [9]:
def prep_calendar(df):
    df = df.drop(["date", "weekday"], axis=1)
    df = df.assign(d = df.d.str[2:].astype(int))
    df = df.fillna("missing")
    cols = list(set(df.columns) - {"wm_yr_wk", "d"})
    df[cols] = OrdinalEncoder(dtype="int").fit_transform(df[cols])
    df = reduce_mem_usage(df)
    return df

calendar = prep_calendar(calendar)

Mem. usage decreased to  0.03 Mb (84.4% reduction)


In [10]:
calendar.head()

,wm_yr_wk,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,11101,0,0,0,1,30,4,4,2,0,0,0
1,11101,1,0,0,2,30,4,4,2,0,0,0
2,11101,2,0,0,3,30,4,4,2,0,0,0
3,11101,3,1,0,4,30,4,4,2,1,1,0
4,11101,4,1,0,5,30,4,4,2,1,0,1


In [11]:
sell_prices.head()

,store_id,item_id,wm_yr_wk,sell_price
0,CA_1,HOBBIES_1_001,11325,9.578125
1,CA_1,HOBBIES_1_001,11326,9.578125
2,CA_1,HOBBIES_1_001,11327,8.257812
3,CA_1,HOBBIES_1_001,11328,8.257812
4,CA_1,HOBBIES_1_001,11329,8.257812


In [12]:
def prep_selling_prices(df):
    gr = df.groupby(["store_id", "item_id"])["sell_price"]
    df["sell_price_rel_diff"] = gr.pct_change()
    df["sell_price_roll_sd7"] = gr.transform(lambda x: x.rolling(7).std())
    df["sell_price_cumrel"] = (gr.shift(0) - gr.cummin()) / (1 + gr.cummax() - gr.cummin())
    df = reduce_mem_usage(df)
    return df

sell_prices = prep_selling_prices(sell_prices)

Mem. usage decreased to 169.63 Mb (0.0% reduction)


In [13]:
sell_prices.tail()

,store_id,item_id,wm_yr_wk,sell_price,sell_price_rel_diff,sell_price_roll_sd7,sell_price_cumrel
6841116,WI_3,FOODS_3_827,11617,1.0,0.0,0.0,0.0
6841117,WI_3,FOODS_3_827,11618,1.0,0.0,0.0,0.0
6841118,WI_3,FOODS_3_827,11619,1.0,0.0,0.0,0.0
6841119,WI_3,FOODS_3_827,11620,1.0,0.0,0.0,0.0
6841120,WI_3,FOODS_3_827,11621,1.0,0.0,0.0,0.0


In [0]:
def reshape_sales(df, drop_d = None):
    if drop_d is not None:
        df = df.drop(["d_" + str(i + 1) for i in range(drop_d)], axis=1)
    df = df.assign(id=df.id.str.replace("_validation", ""))
    df = df.reindex(columns=df.columns.tolist() + ["d_" + str(1913 + i + 1) for i in range(2 * 28)])
    df = df.melt(id_vars=["id", "item_id", "dept_id", "cat_id", "store_id", "state_id"],
                 var_name='d', value_name='demand')
    df = df.assign(d=df.d.str[2:].astype("int16"))
    return df


sales_train_validation = reshape_sales(sales_train_validation, 1000)

In [15]:
sales_train_validation.tail()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand
29544805,FOODS_3_823_WI_3,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,1969,NaN
29544806,FOODS_3_824_WI_3,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,1969,NaN
29544807,FOODS_3_825_WI_3,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,1969,NaN
29544808,FOODS_3_826_WI_3,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,1969,NaN
29544809,FOODS_3_827_WI_3,FOODS_3_827,FOODS_3,FOODS,WI_3,WI,1969,NaN


In [16]:
def prep_sales(df):
    df['lag_t7'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(7))
    df['lag_t28'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28))
    df['rolling_mean_t77'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(7).rolling(7).mean())
    df['rolling_mean_t28'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(7).rolling(28).mean())
    df['rolling_mean_t287'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).mean())
    df['rolling_mean_t2828'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(28).mean())
    df['rolling_median_t287'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).median())
    df['rolling_median_t2828'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(28).median())
    df['rolling_std_t287'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(7).std())
    df['rolling_std_t2828'] = df.groupby(['id'])['demand'].transform(lambda x: x.shift(28).rolling(28).std())

    # Remove rows with NAs except for submission rows. rolling_mean_t180 was selected as it produces most missings
    df = df[(df.d >= 1914) | (pd.notna(df.rolling_mean_t2828))]
    df = reduce_mem_usage(df)


    return df

sales_train_validation = prep_sales(sales_train_validation)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Mem. usage decreased to 2126.15 Mb (45.2% reduction)


In [17]:
sales_train_validation.head()


,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_t7,lag_t28,rolling_mean_t77,rolling_mean_t28,rolling_mean_t287,rolling_mean_t2828,rolling_median_t287,rolling_median_t2828,rolling_std_t287,rolling_std_t2828
1676950,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1056,2.0,3.0,0.0,1.000000,0.643066,0.142822,0.714355,0.0,1.0,0.377930,0.810059
1676951,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1056,0.0,0.0,0.0,0.428467,0.178589,0.285645,0.142822,0.0,0.0,0.488037,0.356445
1676952,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1056,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
1676953,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1056,1.0,11.0,1.0,2.857422,2.714844,1.571289,2.072266,2.0,2.0,1.397461,1.960938
1676954,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1056,1.0,0.0,2.0,0.571289,0.856934,1.000000,0.678711,1.0,0.0,1.154297,0.904785


In [18]:
sales_train_validation = sales_train_validation.merge(calendar, how="left", on="d")
gc.collect()
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_t7,lag_t28,rolling_mean_t77,rolling_mean_t28,rolling_mean_t287,rolling_mean_t2828,rolling_median_t287,rolling_median_t2828,rolling_std_t287,rolling_std_t2828,wm_yr_wk,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1056,2.0,3.0,0.0,1.000000,0.643066,0.142822,0.714355,0.0,1.0,0.377930,0.810059,11347,5,11,2,30,4,4,2,0,0,0
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1056,0.0,0.0,0.0,0.428467,0.178589,0.285645,0.142822,0.0,0.0,0.488037,0.356445,11347,5,11,2,30,4,4,2,0,0,0
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1056,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,11347,5,11,2,30,4,4,2,0,0,0
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1056,1.0,11.0,1.0,2.857422,2.714844,1.571289,2.072266,2.0,2.0,1.397461,1.960938,11347,5,11,2,30,4,4,2,0,0,0
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1056,1.0,0.0,2.0,0.571289,0.856934,1.000000,0.678711,1.0,0.0,1.154297,0.904785,11347,5,11,2,30,4,4,2,0,0,0


In [19]:
sales_train_validation = sales_train_validation.merge(sell_prices, how="left", on=["wm_yr_wk", "store_id", "item_id"])
sales_train_validation.drop(["wm_yr_wk"], axis=1, inplace=True)
gc.collect()
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_t7,lag_t28,rolling_mean_t77,rolling_mean_t28,rolling_mean_t287,rolling_mean_t2828,rolling_median_t287,rolling_median_t2828,rolling_std_t287,rolling_std_t2828,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,sell_price_rel_diff,sell_price_roll_sd7,sell_price_cumrel
0,HOBBIES_1_001_CA_1,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,1056,2.0,3.0,0.0,1.000000,0.643066,0.142822,0.714355,0.0,1.0,0.377930,0.810059,5,11,2,30,4,4,2,0,0,0,8.257812,0.0,0.0,0.000000
1,HOBBIES_1_002_CA_1,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,1056,0.0,0.0,0.0,0.428467,0.178589,0.285645,0.142822,0.0,0.0,0.488037,0.356445,5,11,2,30,4,4,2,0,0,0,3.970703,0.0,0.0,0.000000
2,HOBBIES_1_003_CA_1,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1056,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,5,11,2,30,4,4,2,0,0,0,NaN,NaN,NaN,NaN
3,HOBBIES_1_004_CA_1,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,1056,1.0,11.0,1.0,2.857422,2.714844,1.571289,2.072266,2.0,2.0,1.397461,1.960938,5,11,2,30,4,4,2,0,0,0,4.640625,0.0,0.0,0.231201
4,HOBBIES_1_005_CA_1,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,1056,1.0,0.0,2.0,0.571289,0.856934,1.000000,0.678711,1.0,0.0,1.154297,0.904785,5,11,2,30,4,4,2,0,0,0,3.080078,0.0,0.0,0.375244


In [20]:
sales_train_validation.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27867860 entries, 0 to 27867859
Data columns (total 32 columns):
 #   Column                Dtype  
---  ------                -----  
 0   id                    object 
 1   item_id               object 
 2   dept_id               object 
 3   cat_id                object 
 4   store_id              object 
 5   state_id              object 
 6   d                     int16  
 7   demand                float16
 8   lag_t7                float16
 9   lag_t28               float16
 10  rolling_mean_t77      float16
 11  rolling_mean_t28      float16
 12  rolling_mean_t287     float16
 13  rolling_mean_t2828    float16
 14  rolling_median_t287   float16
 15  rolling_median_t2828  float16
 16  rolling_std_t287      float16
 17  rolling_std_t2828     float16
 18  wday                  int8   
 19  month                 int8   
 20  year                  int8   
 21  event_name_1          int8   
 22  event_type_1          int8   
 23  event

In [21]:
cat_id_cols = ["item_id", "dept_id", "store_id", "cat_id", "state_id"]
cat_cols = cat_id_cols + ["wday", "month", "year", "event_name_1", 
                          "event_type_1", "event_name_2", "event_type_2"]

# In loop to minimize memory use
for i, v in tqdm(enumerate(cat_id_cols)):
    sales_train_validation[v] = OrdinalEncoder(dtype="int").fit_transform(sales_train_validation[[v]])

sales_train_validation = reduce_mem_usage(sales_train_validation)
sales_train_validation.head()
gc.collect()


Mem. usage decreased to 1700.92 Mb (34.7% reduction)


0

In [23]:
num_cols = ["sell_price", "sell_price_rel_diff", "sell_price_roll_sd7", "sell_price_cumrel","lag_t7",
            "lag_t28", "rolling_mean_t77", "rolling_mean_t28", "rolling_mean_t287", 
            "rolling_mean_t2828", "rolling_median_t287", "rolling_median_t2828", "rolling_std_t287","rolling_std_t2828"]
bool_cols = ["snap_CA", "snap_TX", "snap_WI"]
dense_cols = num_cols + bool_cols

# Need to do column by column due to memory constraints
for i, v in tqdm(enumerate(num_cols)):
    sales_train_validation[v] = sales_train_validation[v].fillna(sales_train_validation[v].median())
    
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_t7,lag_t28,rolling_mean_t77,rolling_mean_t28,rolling_mean_t287,rolling_mean_t2828,rolling_median_t287,rolling_median_t2828,rolling_std_t287,rolling_std_t2828,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,sell_price_rel_diff,sell_price_roll_sd7,sell_price_cumrel
0,HOBBIES_1_001_CA_1,1437,3,1,0,0,1056,2.0,3.0,0.0,1.000000,0.643066,0.142822,0.714355,0.0,1.0,0.377930,0.810059,5,11,2,30,4,4,2,0,0,0,8.257812,0.0,0.0,0.000000
1,HOBBIES_1_002_CA_1,1438,3,1,0,0,1056,0.0,0.0,0.0,0.428467,0.178589,0.285645,0.142822,0.0,0.0,0.488037,0.356445,5,11,2,30,4,4,2,0,0,0,3.970703,0.0,0.0,0.000000
2,HOBBIES_1_003_CA_1,1439,3,1,0,0,1056,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,5,11,2,30,4,4,2,0,0,0,3.480469,0.0,0.0,0.000000
3,HOBBIES_1_004_CA_1,1440,3,1,0,0,1056,1.0,11.0,1.0,2.857422,2.714844,1.571289,2.072266,2.0,2.0,1.397461,1.960938,5,11,2,30,4,4,2,0,0,0,4.640625,0.0,0.0,0.231201
4,HOBBIES_1_005_CA_1,1441,3,1,0,0,1056,1.0,0.0,2.0,0.571289,0.856934,1.000000,0.678711,1.0,0.0,1.154297,0.904785,5,11,2,30,4,4,2,0,0,0,3.080078,0.0,0.0,0.375244


In [24]:
test = sales_train_validation[sales_train_validation.d >= 1914]
test = test.assign(id=test.id + "_" + np.where(test.d <= 1941, "validation", "evaluation"),
                   F="F" + (test.d - 1913 - 28 * (test.d > 1941)).astype("str"))
test.head()
gc.collect()

18

In [25]:
sales_train_validation.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d,demand,lag_t7,lag_t28,rolling_mean_t77,rolling_mean_t28,rolling_mean_t287,rolling_mean_t2828,rolling_median_t287,rolling_median_t2828,rolling_std_t287,rolling_std_t2828,wday,month,year,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI,sell_price,sell_price_rel_diff,sell_price_roll_sd7,sell_price_cumrel
0,HOBBIES_1_001_CA_1,1437,3,1,0,0,1056,2.0,3.0,0.0,1.000000,0.643066,0.142822,0.714355,0.0,1.0,0.377930,0.810059,5,11,2,30,4,4,2,0,0,0,8.257812,0.0,0.0,0.000000
1,HOBBIES_1_002_CA_1,1438,3,1,0,0,1056,0.0,0.0,0.0,0.428467,0.178589,0.285645,0.142822,0.0,0.0,0.488037,0.356445,5,11,2,30,4,4,2,0,0,0,3.970703,0.0,0.0,0.000000
2,HOBBIES_1_003_CA_1,1439,3,1,0,0,1056,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,5,11,2,30,4,4,2,0,0,0,3.480469,0.0,0.0,0.000000
3,HOBBIES_1_004_CA_1,1440,3,1,0,0,1056,1.0,11.0,1.0,2.857422,2.714844,1.571289,2.072266,2.0,2.0,1.397461,1.960938,5,11,2,30,4,4,2,0,0,0,4.640625,0.0,0.0,0.231201
4,HOBBIES_1_005_CA_1,1441,3,1,0,0,1056,1.0,0.0,2.0,0.571289,0.856934,1.000000,0.678711,1.0,0.0,1.154297,0.904785,5,11,2,30,4,4,2,0,0,0,3.080078,0.0,0.0,0.375244


In [26]:
sales_train_validation.store_id.unique()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=int8)

In [27]:
# Input dict for training with a dense array and separate inputs for each embedding input
def make_X(df):
    X = {"dense1": df[dense_cols].to_numpy()}
    for i, v in enumerate(cat_cols):
        X[v] = df[[v]].to_numpy()
    return X

# Submission data
X_test = make_X(test)

# One month of validation data
flag = (sales_train_validation.d < 1914) & (sales_train_validation.d >= 1914 - 28)
valid = (make_X(sales_train_validation[flag]),
         sales_train_validation["demand"][flag])

# Rest is used for training
flag = sales_train_validation.d < 1914 - 28
X_train = make_X(sales_train_validation[flag])
y_train = sales_train_validation["demand"][flag]
                             
del sales_train_validation,flag
gc.collect()

0

In [0]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dense, Input, Embedding, Dropout, concatenate, Flatten, LSTM
from tensorflow.keras.models import Model

In [0]:
def create_model(lr=0.002):
    tf.random.set_seed(173)

    tf.keras.backend.clear_session()
    gc.collect()

    # Dense input
    dense_input = Input(shape=(len(dense_cols), ), name='dense1')

    # Embedding input
    wday_input = Input(shape=(1,), name='wday')
    month_input = Input(shape=(1,), name='month')
    year_input = Input(shape=(1,), name='year')
    event_name_1_input = Input(shape=(1,), name='event_name_1')
    event_type_1_input = Input(shape=(1,), name='event_type_1')
    event_name_2_input = Input(shape=(1,), name='event_name_2')
    event_type_2_input = Input(shape=(1,), name='event_type_2')
    item_id_input = Input(shape=(1,), name='item_id')
    dept_id_input = Input(shape=(1,), name='dept_id')
    store_id_input = Input(shape=(1,), name='store_id')
    cat_id_input = Input(shape=(1,), name='cat_id')
    state_id_input = Input(shape=(1,), name='state_id')

    wday_emb = Flatten()(Embedding(7, 1)(wday_input))
    month_emb = Flatten()(Embedding(12, 1)(month_input))
    year_emb = Flatten()(Embedding(6, 1)(year_input))
    event_name_1_emb = Flatten()(Embedding(31, 1)(event_name_1_input))
    event_type_1_emb = Flatten()(Embedding(5, 1)(event_type_1_input))
    event_name_2_emb = Flatten()(Embedding(5, 1)(event_name_2_input))
    event_type_2_emb = Flatten()(Embedding(5, 1)(event_type_2_input))

    item_id_emb = Flatten()(Embedding(3049, 3)(item_id_input))
    dept_id_emb = Flatten()(Embedding(7, 1)(dept_id_input))
    store_id_emb = Flatten()(Embedding(10, 1)(store_id_input))
    cat_id_emb = Flatten()(Embedding(3, 1)(cat_id_input))
    state_id_emb = Flatten()(Embedding(3, 1)(state_id_input))

    # Combine dense and embedding parts and add dense layers. Exit on linear scale.
    x = concatenate([dense_input, wday_emb, month_emb, year_emb, 
                     event_name_1_emb, event_type_1_emb, 
                     event_name_2_emb, event_type_2_emb, 
                     item_id_emb, dept_id_emb, store_id_emb,
                     cat_id_emb, state_id_emb])
    x = Dense(256, activation="relu")(x)
    x = Dense(128, activation="relu")(x)
    x = Dense(64, activation="relu")(x)
    x = Dense(16, activation="relu")(x)
    x = Dense(4, activation="relu")(x)
    outputs = Dense(1, activation="linear", name='output')(x)

    inputs = {"dense1": dense_input, "wday": wday_input, "month": month_input, "year": year_input, 
              "event_name_1": event_name_1_input, "event_type_1": event_type_1_input,
              "event_name_2": event_name_2_input, "event_type_2": event_type_2_input,
              "item_id": item_id_input, "dept_id": dept_id_input, "store_id": store_id_input, 
              "cat_id": cat_id_input, "state_id": state_id_input}

    # Connect input and output
    model = Model(inputs, outputs)

    model.compile(loss=keras.losses.mean_squared_error,
                  metrics=["mse"],
                  optimizer=keras.optimizers.RMSprop(learning_rate=lr))
    return model

In [32]:
model = create_model(0.0002)
model.summary()


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
wday (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
month (InputLayer)              [(None, 1)]          0                                            
__________________________________________________________________________________________________
year (InputLayer)               [(None, 1)]          0                                            
__________________________________________________________________________________________________
event_name_1 (InputLayer)       [(None, 1)]          0                                            
______________________________________________________________________________________________

In [0]:
from keras.callbacks import ModelCheckpoint

In [0]:
checkpointer = ModelCheckpoint(filepath='best_weights.hdf5',monitor='val_accuracy',verbose=1,save_best_only=True)

In [39]:
history = model.fit(X_train, 
                    y_train,
                    batch_size=10000,
                    epochs=70,
                    # callbacks=[checkpointer],
                    shuffle=True,
                    validation_data=valid)

Epoch 1/70
2531/2531 [==============================] - 26s 10ms/step - loss: 5.0502 - mse: 5.0504 - val_loss: 4.2655 - val_mse: 4.2617
Epoch 2/70
2531/2531 [==============================] - 25s 10ms/step - loss: 4.8635 - mse: 4.8637 - val_loss: 4.0876 - val_mse: 4.0825
Epoch 3/70
2531/2531 [==============================] - 26s 10ms/step - loss: 4.7955 - mse: 4.7956 - val_loss: 4.0926 - val_mse: 4.0865
Epoch 4/70
2531/2531 [==============================] - 26s 10ms/step - loss: 4.7553 - mse: 4.7552 - val_loss: 4.2374 - val_mse: 4.2330
Epoch 5/70
2531/2531 [==============================] - 25s 10ms/step - loss: 4.7184 - mse: 4.7183 - val_loss: 4.0800 - val_mse: 4.0751
Epoch 6/70
2531/2531 [==============================] - 26s 10ms/step - loss: 4.6978 - mse: 4.6979 - val_loss: 4.2038 - val_mse: 4.1974
Epoch 7/70
2531/2531 [==============================] - 25s 10ms/step - loss: 4.6680 - mse: 4.6682 - val_loss: 4.1187 - val_mse: 4.1147
Epoch 8/70
2531/2531 [==========================

KeyboardInterrupt: ignored

In [40]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

NameError: ignored

In [0]:

# model.load_weights('best_weights.hdf5')
model.save('model.h5')

In [0]:
pred = model.predict(X_test, batch_size=10000)

In [43]:
test["demand"] = pred.clip(0)
submission_final = test.pivot(index="id", columns="F", values="demand").reset_index()[submission.columns]
submission = submission[["id"]].merge(submission_final, how="left", on="id")
submission.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,F11,F12,F13,F14,F15,F16,F17,F18,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.715907,0.668565,0.661241,0.725497,0.782398,1.063998,1.150220,0.497051,0.521660,0.466258,0.436803,0.549465,0.734568,0.567438,0.506822,0.452690,0.490818,0.545881,0.625249,0.825639,0.787772,0.478691,0.466657,0.460625,0.472919,0.505861,0.754950,0.777849
1,HOBBIES_1_002_CA_1_validation,0.145434,0.149990,0.194897,0.171061,0.182613,0.238234,0.240779,0.254460,0.236098,0.243629,0.267353,0.318453,0.398378,0.548260,0.256230,0.245270,0.261095,0.275755,0.324192,0.408343,0.443698,0.252550,0.238383,0.239565,0.235084,0.278841,0.366696,0.386032
2,HOBBIES_1_003_CA_1_validation,0.454159,0.444647,0.432956,0.453726,0.507856,0.429290,0.422333,0.235432,0.205464,0.204334,0.239428,0.289170,0.320807,0.384824,0.224294,0.204353,0.213220,0.235689,0.335812,0.416601,0.429937,0.296384,0.285227,0.287926,0.311515,0.392797,0.437998,0.431223
3,HOBBIES_1_004_CA_1_validation,1.728431,1.567787,1.468926,1.561896,2.047310,2.539503,2.477524,0.761232,0.655411,0.717950,0.670500,0.949704,1.198598,1.150455,0.707197,0.718599,0.671192,0.668927,0.814420,1.203449,1.301968,0.790347,0.699138,0.659309,0.676767,0.824197,1.221007,1.054677
4,HOBBIES_1_005_CA_1_validation,0.961969,0.948024,0.868872,1.038014,1.208967,1.512400,1.591452,0.503738,0.462052,0.491882,0.543025,0.637081,0.828913,0.849981,0.570611,0.576433,0.527298,0.545395,0.731433,1.001058,0.927498,0.617097,0.562041,0.548580,0.584412,0.707422,0.995538,0.843051


In [0]:
submission.to_csv("submission.csv", index=False)